In [4]:
!pip install neo4j spacy rapidfuzz

  Using cached numpy-2.0.2-cp311-cp311-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.0.2-cp311-cp311-win_amd64.whl (15.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\Nasiba\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 8.5 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 6.1 MB/s eta 0:00:02
     ------------------ --------------------- 5.8/12.8 MB 10.1 MB/s eta 0:00:01
     ----------------------------- ---------- 9.4/12.8 MB 11.7 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 12.7 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 10.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\Nasiba\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:

from neo4j import GraphDatabase
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "testmaster123"))


In [5]:
with driver.session() as s:
    s.run("CREATE CONSTRAINT ent_name IF NOT EXISTS FOR (n:Entity) REQUIRE n.name IS UNIQUE")
    s.run("CREATE CONSTRAINT doc_id IF NOT EXISTS FOR (n:Document) REQUIRE n.id IS UNIQUE")
    s.run("CREATE FULLTEXT INDEX doc_text_fts IF NOT EXISTS FOR (d:Document) ON EACH [d.text]")
print("Schema erstellt.")


Schema erstellt.


In [11]:
with driver.session() as s:
    s.run("MATCH (n:Entity) DETACH DELETE n")
    s.run("MATCH (n:Document) DETACH DELETE n")
print("Alte Demo-Knoten entfernt.")


Alte Demo-Knoten entfernt.


In [2]:
cypher_load = """
CALL apoc.load.json('file:///docling_chunks.jsonl')
YIELD value AS v
WITH v
MERGE (d:Document {id: coalesce(v.chunk_id, v.id)})
SET d.text     = v.text,
    d.product  = v.product,
    d.category = v.category,
    d.doc_type = v.chunk_type,
    d.source   = coalesce(v.source, v.path),
    d.page     = v.page,
    d.section  = v.section
"""
with driver.session() as s:
    s.run(cypher_load)
print("JSONL geladen → Document-Knoten erstellt.")


JSONL geladen → Document-Knoten erstellt.


In [3]:


cypher_load = """
MATCH (d:Document)
WHERE d.category = "Portenta Family"
MERGE (c:Category {name: d.category})
MERGE (p:Product {name: d.product})
MERGE (p)-[:BELONGS_TO]->(c)
MERGE (p)-[:DESCRIBED_IN]->(d);
"""
with driver.session() as s:
    s.run(cypher_load)
print("JSONL geladen → Document-Knoten erstellt.")

JSONL geladen → Document-Knoten erstellt.


In [8]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","testmaster123"))

with driver.session() as s:
    # Constraints (sicher einzeln ausführen)
    s.run("CREATE CONSTRAINT IF NOT EXISTS FOR (n:Category) REQUIRE n.name IS UNIQUE")
    s.run("CREATE CONSTRAINT IF NOT EXISTS FOR (n:Product)  REQUIRE n.name IS UNIQUE")

    # Verknüpfungen anlegen
    s.run("""
    MATCH (d:Document)
    WHERE d.product IS NOT NULL AND d.category IS NOT NULL
    MERGE (c:Category {name:d.category})
    MERGE (p:Product  {name:d.product})
    MERGE (p)-[:BELONGS_TO]->(c)
    MERGE (p)-[:DESCRIBED_IN]->(d)
    """)
print("Category/Product/Dokument verbunden.")


Category/Product/Dokument verbunden.


In [16]:
import re

INTERFACES = {"i2c","spi","uart","pwm","analog","digital","can","ethernet","usb","wifi","bluetooth","ble"}
spec_patterns = [
    r"(operating voltage)\s*[:\-]?\s*([\d\.,]+)\s*([a-zA-Z°]+)?",
    r"(input voltage)\s*[:\-]?\s*([\d\.,]+)\s*([a-zA-Z°]+)?",
    r"(clock(?: speed)?)\s*[:\-]?\s*([\d\.,]+)\s*(mhz|khz|hz)",
    r"(analog inputs?)\s*[:\-]?\s*([\d]+)",
    r"(digital (?:i\/o|pins?))\s*[:\-]?\s*([\d]+)",
]
UNIT_MAP = {"v":"V","mv":"mV","a":"A","ma":"mA","hz":"Hz","khz":"kHz","mhz":"MHz","°c":"°C","c":"°C"}

def norm_unit(u): 
    if not u: return ""
    u = u.strip().lower()
    return UNIT_MAP.get(u, u)

with driver.session() as s:
    # Interfaces
    s.run("""
    MATCH (p:Product)-[:DESCRIBED_IN]->(d:Document)
    WITH p,d, toLower(d.text) AS txt
    WITH p,d, [iface IN $ifaces WHERE txt =~ ('.*\\b'+iface+'\\b.*')] AS hits
    UNWIND hits AS iface
    MERGE (i:Interface {name: toUpper(iface)})
    MERGE (p)-[r:SUPPORTS_INTERFACE]->(i)
    ON CREATE SET r.source='text_rule', r.confidence=0.8, r.evidence=[d.id]
    ON MATCH  SET r.evidence=apoc.coll.toSet(coalesce(r.evidence,[])+d.id),
                 r.confidence=CASE WHEN r.confidence<0.8 THEN 0.8 ELSE r.confidence END
    """, {"ifaces": list(INTERFACES)})

    # Specs
    for pat in spec_patterns:
        query = """
    MATCH (p:Product)-[:DESCRIBED_IN]->(d:Document)
    WITH p,d, apoc.text.regexGroups(d.text, $pat) AS matches
    UNWIND matches AS value
    // value[0] = gesamter Treffer; Gruppen beginnen bei 1: (key, value, unit?)
    WITH p,d,
         apoc.text.capitalizeAll(trim(value[1]))              AS k,
         trim(value[2])                                       AS v,
         trim(coalesce(value[3], ''))                         AS u
    MERGE (s:Spec {id: p.name + ':' + k + ':' + v + ':' + u})
    ON CREATE SET s.key = k, s.value = v, s.unit = u
    MERGE (p)-[:HAS_SPEC]->(s)
    """
        with driver.session() as s:
            s.run(query, {"pat": pat})

    print("Regelbasierte Kanten/Specs erzeugt.")






Regelbasierte Kanten/Specs erzeugt.


In [17]:
import spacy
from rapidfuzz import fuzz

nlp = spacy.load("en_core_web_sm")
TRIGGERS = ["use", "uses", "used with", "include", "includes", "compatible with", "supports", "secure element", "co-processor"]

# kleine Gazetteer für bekannte Komponenten (kannst du erweitern)
COMPONENT_ALIASES = {
    "se050c2": "SE050C2 IoT Secure Element",
    "se050": "SE050C2 IoT Secure Element",
    "nina-w10": "u-blox NINA-W10",
}

def canonical_component(toktext: str) -> str|None:
    t = toktext.lower()
    best,score = None,0
    for alias,canon in COMPONENT_ALIASES.items():
        sc = fuzz.partial_ratio(alias, t)
        if sc>score:
            best,score = canon, sc
    return best if score>=85 else None

# hole Dokumente aus Neo4j, verarbeite in Python, schreibe Kanten zurück
with driver.session() as s:
    docs = s.run("""
    MATCH (p:Product)-[:DESCRIBED_IN]->(d:Document)
    RETURN p.name AS product, d.id AS cid, d.text AS text
    """).data()

for row in docs:
    product, cid, text = row["product"], row["cid"], row["text"] or ""
    if not text: continue
    doc = nlp(text)

    # 1) Trigger vorhanden?
    txt_low = text.lower()
    if not any(t in txt_low for t in TRIGGERS):
        continue

    # 2) Entitäten-Kandidaten (PROPN/ORG/PRODUCT)
    cand_names = set()
    for ent in doc.ents:
        if ent.label_ in {"ORG","PRODUCT"}:
            cand_names.add(ent.text)
    # zusätzlich simple NounChunks aufnehmen
    for nc in doc.noun_chunks:
        if any(c.isupper() for c in nc.text if c.isalpha()) and len(nc.text) <= 40:
            cand_names.add(nc.text)

    # 3) Canonical-Mapping + Eintrag
    for name in cand_names:
        comp = canonical_component(name)
        if not comp:
            continue
        with driver.session() as s:
            s.run("""
            MERGE (p:Product {name:$product})
            MERGE (c:Component {name:$comp})
            MERGE (p)-[r:CANDIDATE_USES]->(c)
            ON CREATE SET r.source='spacy+triggers', r.confidence=0.7, r.evidence=[$cid]
            ON MATCH  SET r.evidence=apoc.coll.toSet(coalesce(r.evidence,[])+$cid),
                         r.confidence=CASE WHEN r.confidence<0.7 THEN 0.7 ELSE r.confidence END
            """, {"product": product, "comp": comp, "cid": cid})

print("spaCy-basierte CANDIDATE_USES erzeugt (wo Trigger & Komponenten erkannt wurden).")


spaCy-basierte CANDIDATE_USES erzeugt (wo Trigger & Komponenten erkannt wurden).
